# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [19]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, precision_score, accuracy_score, recall_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [20]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [21]:
#your code here
spaceship.dropna(inplace=True)
spaceship["Cabin"] = spaceship["Cabin"].apply(lambda x: x.split("/")[0])
spaceship.drop(columns = ["PassengerId","Name"], inplace=True)

spaceship2 = spaceship.__deepcopy__()
spacedummies = pd.get_dummies(spaceship2, columns=['HomePlanet', 'Cabin', 'Destination'], drop_first=False)
spacedummies = pd.get_dummies(spacedummies, columns=['CryoSleep', 'VIP', 'Transported'], drop_first=True)

In [22]:
target = spacedummies["Transported_True"]
features = spacedummies.drop("Transported_True", axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [23]:
#your code here
binary_features = ['HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars', 
                  'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 
                  'Cabin_F', 'Cabin_G', 'Cabin_T', 
                  'Destination_55 Cancri e', 'Destination_PSO J318.5-22', 
                  'Destination_TRAPPIST-1e', 'CryoSleep_True', 'VIP_True']

numerical_features = X_train.columns.difference(binary_features) # Identify numerical features

# Normalize only the numerical features
normalizer = MinMaxScaler()
normalizer.fit(X_train[numerical_features])

# Transform the numerical features
X_train_norm = normalizer.transform(X_train[numerical_features])
X_test_norm = normalizer.transform(X_test[numerical_features])

# Convert to DataFrame, keeping the original indices
X_train_norm = pd.DataFrame(X_train_norm, columns=numerical_features, index=X_train.index)
X_test_norm = pd.DataFrame(X_test_norm, columns=numerical_features, index=X_test.index)

# Combine normalized numerical features with unchanged binary features
X_train_norm = pd.concat([X_train_norm, X_train[binary_features]], axis=1)
X_test_norm = pd.concat([X_test_norm, X_test[binary_features]], axis=1)

In [24]:
# Picking GradientBoost as our best performer

gb_reg = GradientBoostingClassifier(max_depth=20,
                                   n_estimators=200)

gb_reg.fit(X_train_norm, y_train)

pred = gb_reg.predict(X_test_norm)

- Evaluate your model

In [25]:
#your code here
precision = precision_score(y_test, pred)
accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.7889561270801816
Precision: 0.7842261904761905
Recall: 0.7972768532526475


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [26]:
#your code here
grid = {"n_estimators": [50, 70, 80, 90],
        "max_leaf_nodes": [70, 80, 85],
        "max_depth":[35,40,45]}

- Run Grid Search

In [27]:
gb_class = GradientBoostingClassifier()
model = GridSearchCV(estimator = gb_class, param_grid = grid, cv=5, n_jobs=-1)

model.fit(X_train_norm, y_train)

display(model.best_params_)

{'max_depth': 40, 'max_leaf_nodes': 70, 'n_estimators': 50}

- Evaluate your model

In [28]:
best_model = model.best_estimator_

pred = best_model.predict(X_test_norm)

precision = precision_score(y_test, pred)
accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.7881996974281392
Precision: 0.7764876632801161
Recall: 0.8093797276853253


In [29]:
grid = {"n_estimators": [30, 50, 70, 80],
        "max_samples" : [750, 800, 900, 1200], 
        "estimator__max_leaf_nodes": [35, 50, 60, 70, 75, 80 , 90],
        "estimator__max_depth":[50,70,90]
        }

In [30]:
bagging_reg = BaggingClassifier(DecisionTreeClassifier(), bootstrap = False, n_jobs=-1)
model = GridSearchCV(estimator = bagging_reg, param_grid = grid, cv=5, n_jobs=-1)

model.fit(X_train_norm, y_train)

display(model.best_params_)

best_model = model.best_estimator_

pred = best_model.predict(X_test_norm)

precision = precision_score(y_test, pred)
accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'estimator__max_depth': 70,
 'estimator__max_leaf_nodes': 35,
 'max_samples': 800,
 'n_estimators': 80}

Accuracy: 0.7874432677760969
Precision: 0.7785923753665689
Recall: 0.8033282904689864
